In [23]:
import numpy as np
import pandas as pd

In [24]:
temp_df = pd.read_csv('IMDB Dataset.csv')

In [4]:
df = temp_df.iloc[:10000]

In [5]:
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [6]:
df.drop_duplicates(inplace=True)

<ipython-input-6-16cdb8520be8>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop_duplicates(inplace=True)


In [7]:
import re
def remove_tags(raw_text):
    cleaned_text = re.sub(re.compile('<.*?>'), '', raw_text)
    return cleaned_text

In [8]:
df['review'] = df['review'].apply(remove_tags)

<ipython-input-8-00e93c2a3043>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['review'] = df['review'].apply(remove_tags)


In [9]:
df['review'] = df['review'].apply(lambda x:x.lower())

<ipython-input-9-afca29351ee0>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['review'] = df['review'].apply(lambda x:x.lower())


In [10]:
from nltk.corpus import stopwords

sw_list = stopwords.words('english')

df['review'] = df['review'].apply(lambda x: [item for item in x.split() if item not in sw_list]).apply(lambda x:" ".join(x))

<ipython-input-10-9f68b5950da1>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['review'] = df['review'].apply(lambda x: [item for item in x.split() if item not in sw_list]).apply(lambda x:" ".join(x))


In [11]:
df['review']

0       one reviewers mentioned watching 1 oz episode ...
1       wonderful little production. filming technique...
2       thought wonderful way spend time hot summer we...
3       basically there's family little boy (jake) thi...
4       petter mattei's "love time money" visually stu...
                              ...                        
9995    fun, entertaining movie wwii german spy (julie...
9996    give break. anyone say "good hockey movie"? kn...
9997    movie bad movie. watching endless series bad h...
9998    movie probably made entertain middle school, e...
9999    smashing film film-making. shows intense stran...
Name: review, Length: 9983, dtype: object

In [12]:
import gensim

In [13]:
from nltk import sent_tokenize
from gensim.utils import simple_preprocess

In [14]:
story = []
for doc in df['review']:
    raw_sent = sent_tokenize(doc)
    for sent in raw_sent:
        story.append(simple_preprocess(sent))
    

In [15]:
model = gensim.models.Word2Vec(
    window=10,
    min_count=2
)

In [16]:
model.build_vocab(story)

In [17]:
model.train(story, total_examples=model.corpus_count, epochs=model.epochs)

(5875535, 6212140)

In [18]:
len(model.wv.index_to_key)

31845

In [19]:
def document_vector(doc):
    # remove out-of-vocabulary words
    doc = [word for word in doc.split() if word in model.wv.index_to_key]
    return np.mean(model.wv[doc], axis=0)

In [20]:
document_vector(df['review'].values[0])

array([-0.13551651,  0.44236022,  0.19618301,  0.2243388 , -0.05095489,
       -0.5747312 ,  0.24217677,  0.9659006 , -0.35396117, -0.20832208,
       -0.2984078 , -0.4400784 ,  0.06137785,  0.0494826 ,  0.17676252,
       -0.12226024,  0.04628523, -0.3924497 , -0.03380001, -0.6410042 ,
        0.04235367,  0.21378344,  0.05942454, -0.29909694, -0.3123422 ,
       -0.02930854, -0.28178704,  0.04482391, -0.32226902,  0.02552933,
        0.3864304 ,  0.02701772,  0.14447899, -0.319503  , -0.16364487,
        0.3655229 ,  0.09109591, -0.46899873, -0.21292035, -0.75319904,
        0.09909733, -0.25865507,  0.06007399, -0.03030884,  0.48151824,
       -0.10978614, -0.25712606,  0.00799898,  0.05369115,  0.36769992,
        0.07791721, -0.3666489 , -0.41765627, -0.11414295, -0.16897762,
        0.24438773,  0.2116644 ,  0.08659696, -0.29066396,  0.08693315,
        0.08008192,  0.12331163, -0.00459293, -0.11629856, -0.48819962,
        0.23666628,  0.02842407,  0.15447702, -0.3748886 ,  0.23

In [21]:
from tqdm import tqdm

In [22]:
X = []
for doc in tqdm(df['review'].values):
    X.append(document_vector(doc))

100%|██████████████████████████████████████████████████████████████████████████████| 9983/9983 [14:20<00:00, 11.59it/s]


In [25]:
X = np.array(X)

In [27]:
X[0]

array([-0.13551651,  0.44236022,  0.19618301,  0.2243388 , -0.05095489,
       -0.5747312 ,  0.24217677,  0.9659006 , -0.35396117, -0.20832208,
       -0.2984078 , -0.4400784 ,  0.06137785,  0.0494826 ,  0.17676252,
       -0.12226024,  0.04628523, -0.3924497 , -0.03380001, -0.6410042 ,
        0.04235367,  0.21378344,  0.05942454, -0.29909694, -0.3123422 ,
       -0.02930854, -0.28178704,  0.04482391, -0.32226902,  0.02552933,
        0.3864304 ,  0.02701772,  0.14447899, -0.319503  , -0.16364487,
        0.3655229 ,  0.09109591, -0.46899873, -0.21292035, -0.75319904,
        0.09909733, -0.25865507,  0.06007399, -0.03030884,  0.48151824,
       -0.10978614, -0.25712606,  0.00799898,  0.05369115,  0.36769992,
        0.07791721, -0.3666489 , -0.41765627, -0.11414295, -0.16897762,
        0.24438773,  0.2116644 ,  0.08659696, -0.29066396,  0.08693315,
        0.08008192,  0.12331163, -0.00459293, -0.11629856, -0.48819962,
        0.23666628,  0.02842407,  0.15447702, -0.3748886 ,  0.23

In [29]:
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()

y = encoder.fit_transform(df['sentiment'])

In [30]:
y

array([1, 1, 1, ..., 0, 0, 1])

In [31]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=1)

In [32]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

In [33]:
rf = RandomForestClassifier()
rf.fit(X_train,y_train)
y_pred = rf.predict(X_test)
accuracy_score(y_test,y_pred)

0.7681522283425137